### Library

In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BusinessMonthEnd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import DateOffset
from tqdm.notebook import tqdm

In [2]:
cross_terms = False
sic_dummies = False
start_date = '1967'

### Raw Data


In [3]:
# Monthly equity return data for all securities on CRSP.
returns = pd.read_csv('crsp_data.csv', parse_dates=['date'])

# 124 macroeconomic predictors from the FRED-MD database as detailed in McCracken and Ng (2016),
# and 8 macroeconomic predictors from Welch and Goyal (2007) which have been suggested as predictors for the equity premium. 
macro = pd.read_csv("MacroTimeSeries.csv")

# Stock characteristics provided by Dacheng Xiu (https://dachxiu.chicagobooth.edu/#risklab)
char = pd.read_csv('datashare.csv', parse_dates=['DATE'])

/Users/xuhuili/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Returns are already adjusted for corporate actions (i.e., dividends, splits, etc.) but we also should adjust for delisting.

In [4]:
returns.columns = map(str.lower, returns.columns)

# deal with missing returns
returns['ret'] = returns['ret'].replace(['B', 'C'], np.nan)
returns['dlret'] = returns['dlret'].replace(['A', 'S', 'T', 'P'], np.nan)
returns['dlret'] = returns['dlret'].astype('float')
returns['ret'] = returns['ret'].astype('float')
returns['dlret'] = returns['dlret'].fillna(0)

# adjust returns for delisting
returns['retadj'] = (1 + returns['ret']) * (1 + returns['dlret']) - 1

In [5]:
# clean macro date
macro['DATE'] = pd.to_datetime(macro['sasdate'], format='%m/%d/%y') # parses two-digit years as "20yy"
future = macro['DATE'] > '2020-01-01' # identify 20th century dates
macro.loc[future,'DATE'] = macro.loc[future,'DATE'] - DateOffset(years=100) # subtract 100 years 
macro = macro.drop(columns='sasdate')
MACRO_1 = macro.iloc[:, 0:124]
MACRO_2 = macro.iloc[:, 170:]
macro = pd.concat([MACRO_1, MACRO_2], axis =1)
macro = macro.set_index('DATE')

### Data Preprocessing

In [6]:
# obtain date, permno pairs for which we have stock characteristics
index = char[['DATE', 'permno']].drop_duplicates()
index['month'] = index['DATE'].dt.month
index['year'] = index['DATE'].dt.year

# return preprocessing from CRSP
returns = returns[['date', 'permno', 'retadj']]
returns['year'] = returns['date'].dt.year
returns['month'] = returns['date'].dt.month

# align returns with stock characteristics
returns = returns.merge(index, on=['year', 'month', 'permno'], how='right')
returns = returns.drop(['year', 'month'], axis=1).set_index(['DATE', 'permno']).dropna()

# obtain date, permno pairs for which we have stock characteristics and CRSP returns
keys = returns.reset_index()[['DATE', 'permno']]
idx = keys.set_index(['DATE', 'permno']).index
del index

# obtain non-duplicate dates from all (date, permno) keys with stock characteristics and CRSP returns
dates = keys[['DATE']].drop_duplicates()
dates['month'] = dates['DATE'].dt.month
dates['year'] = dates['DATE'].dt.year

# align macro data
macro['month'] = macro.index.month
macro['year'] = macro.index.year
macro = macro.reset_index()
macro = macro.merge(dates, on=['month', 'year'], how='left')
macro = macro.drop(['year', 'month'], axis=1)
macro['DATE'] = macro['DATE_y']
macro['DATE'] = macro['DATE'].fillna(macro['DATE_x'])
macro = macro.drop(['DATE_x', 'DATE_y'], axis=1).set_index('DATE')

# align stock characteristics
char = char.set_index(['DATE', 'permno']).loc[idx]

### Excess Returns

To calculate excess returns, we also obtain the Treasury-bill rate. Following Welch and Goyal (2008), we use the 3-month rate. Welch and Goyal obtain the 3-Month Treasury Bill: Secondary Market Rate from the economic research database at the Federal Reserve Bank at St. Louis (FRED) as the average of the corresponding daily series. They use the rate from the beginning of the period to calculate excess returns at the end of the period since the risk-free rate is known in advance.

In [7]:
rf = macro[['tbl']].shift(-1)/12
tds = returns.reset_index().merge(rf, on=['DATE'], how='left')
tds['eret'] = tds['retadj'] - tds['tbl']
excess_returns = tds.set_index(['DATE', 'permno'])['eret']
del returns

### Features - Stock Characteristics

Missing values in stock characteristics are filled with the cross-sectional median. Some features do not have any data before a certain time, which results in missing values even after filling missing values with the cross-sectional median. These missing values are filled with zeros.

Then, we cross-sectionally rank stock characteristics and map them to (-1,1) using the following formula:

$$f^*_{i,t} = \frac{2}{N+1}\times CSrank(f_{i,t})-1$$


This transformation makes sure that at each point in time all features are cross-sectionally demeaned. As a result, all features in the training sample are also cross-sectionally demeaned.

In [ ]:
char

### Fill missing values with cross-sectional median

In [9]:
# obtain sic dummies
# sic = pd.get_dummies(char, columns=['sic2'], dummy_na=False)

# obtain industry labels
sic2 = char['sic2']

#drop sic column from stock characteristics
char = char.drop('sic2', axis=1)

# fill missing values with cross-sectional median
xsectional = char.groupby(level=0).transform('median')
char = char.fillna(xsectional)

In [10]:
char

,,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,mom6m,mom12m,...,stdacc,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade
DATE,permno,,,,,,,,,,,,,,,,,,,,,
1986-02-28,10000,1.610000e+04,0.951610,0.910403,0.000553,9.822501,0.050832,0.208637,0.017991,0.036933,0.096988,...,0.119550,0.121799,3.0,0.076998,1.244051e-06,0.250000,0.065278,1.231289,2.120805,4.785175e-08
1986-03-31,10000,1.196000e+04,0.948993,0.914230,0.102942,9.824918,0.050760,0.255908,-0.257143,0.071718,0.090909,...,0.119881,0.121946,3.0,0.055511,1.891760e-06,0.044776,0.031004,1.021089,1.079774,1.023392e-07
1986-04-30,10000,1.633000e+04,0.939002,0.895257,0.255166,7.897668,0.050834,0.368892,0.365385,0.192875,0.162667,...,0.120066,0.122603,3.0,0.037231,7.315091e-07,0.145161,0.044548,1.033817,1.745333,7.467463e-08
1986-05-30,10000,1.517200e+04,0.949946,0.915980,0.203376,8.472954,0.050677,0.388370,-0.098592,0.222509,0.238449,...,0.121684,0.124578,3.0,0.048336,1.215981e-06,0.022727,0.011246,1.184555,1.502285,7.649551e-08
1986-06-30,10000,1.179386e+04,0.947107,0.912578,0.156985,8.250098,0.050402,0.400748,-0.222656,0.171429,0.217391,...,0.122030,0.124959,3.0,0.062245,2.744328e-06,0.115702,0.038863,0.959128,1.756198,7.360224e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-08-31,93436,3.491163e+07,1.600823,2.562634,0.509589,18.685227,0.053493,-0.101592,0.106039,0.110251,-0.202405,...,0.318933,0.991338,4.0,0.027501,1.507552e-11,0.036904,0.016816,0.274125,7.100033,4.363933e-09
2016-09-30,93436,3.164016e+07,1.636403,2.677816,0.334004,18.492052,0.053109,0.017851,-0.097023,0.223311,-0.057295,...,0.318933,0.991338,4.0,0.020614,1.517188e-11,0.021347,0.010832,0.334028,6.298895,4.550212e-09
2016-10-31,93436,3.056879e+07,1.633774,2.669218,-0.037025,18.518768,0.052457,0.128428,-0.037640,-0.077295,-0.146498,...,0.318933,0.991338,4.0,0.026596,2.033979e-11,0.025533,0.019162,0.321573,9.232955,4.396939e-09


### Cross-sectional ranking

In [11]:
def rank(tds):
    ind = tds[['DATE', 'permno']]
    tds = tds.drop(['DATE', 'permno'], axis=1)
    tds = tds.rank(axis=0)
    N = tds.count(numeric_only=True)
    for col in tds:
        tds[col] = tds[col]*(2/(N[col]+1))-1
    tds = pd.concat([ind, tds], axis=1)
    return tds

# rank characteristics cross-sectionally
char = char.reset_index().groupby(['DATE']).apply(rank)

# fill remaining missing values with zeros
char = char.fillna(0)
char = char.set_index(['DATE', 'permno'])

### Merge Features

In [12]:
excess_returns = excess_returns.to_frame()

In [13]:
df = pd.merge(excess_returns, char, left_index=True, right_index=True)
df = pd.merge(df, sic2, left_index=True, right_index=True)
df = df.dropna()

data = pd.merge(df, macro, left_index=True, right_index=True)
data.reset_index(level=1, inplace=True)
data = data.drop_duplicates()

In [14]:
data

,permno,eret,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,INVEST,VXOCLSx,dp,ep_y,b/m,ntis,tbl,tms,dfy,svar
DATE,,,,,,,,,,,,,,,,,,,,,
1963-07-31,10006,-0.047811,0.582133,-0.745437,-0.745437,-0.428434,0.707012,-0.647454,0.935159,0.197887,...,90.0224,12.0403,-3.450997,-2.893982,0.567282,0.018263,0.0299,0.0108,0.0061,0.000261
1963-07-31,10014,-0.002650,-0.381364,-0.757925,-0.757925,-0.532181,-0.409222,0.853026,-0.985110,-0.680596,...,90.0224,12.0403,-3.450997,-2.893982,0.567282,0.018263,0.0299,0.0108,0.0061,0.000261
1963-07-31,10102,-0.018929,0.734870,0.455331,0.455331,-0.535062,0.793468,-0.758886,0.826129,-0.362152,...,90.0224,12.0403,-3.450997,-2.893982,0.567282,0.018263,0.0299,0.0108,0.0061,0.000261
1963-07-31,10137,-0.002650,0.836695,-0.802113,-0.802113,-0.551393,0.488953,-0.849183,0.724784,0.652257,...,90.0224,12.0403,-3.450997,-2.893982,0.567282,0.018263,0.0299,0.0108,0.0061,0.000261
1963-07-31,10145,0.020546,0.959654,-0.472622,-0.472622,-0.402498,0.931796,-0.762728,0.504323,-0.148895,...,90.0224,12.0403,-3.450997,-2.893982,0.567282,0.018263,0.0299,0.0108,0.0061,0.000261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-11-30,93428,-0.001578,0.330875,0.669828,0.667369,-0.614682,0.415876,0.179487,-0.177028,-0.613980,...,3325.6514,14.7328,-3.849851,-3.152198,0.319688,-0.028997,0.0033,0.0187,0.0087,0.000364
2016-11-30,93429,0.093598,0.716192,-0.790657,-0.800492,0.302072,0.731647,-0.469968,-0.506674,0.108184,...,3325.6514,14.7328,-3.849851,-3.152198,0.319688,-0.028997,0.0033,0.0187,0.0087,0.000364
2016-11-30,93433,0.278695,-0.879171,0.902705,0.900597,-0.069898,-0.740429,0.993502,-0.486126,-0.829996,...,3325.6514,14.7328,-3.849851,-3.152198,0.319688,-0.028997,0.0033,0.0187,0.0087,0.000364


In [15]:
print(len(data['permno'].unique()))

26641


### Dataset Timeframe

Setting the timeframe of our dataset from 1966/10/31to 2016/11/30, which is 50 years in total. 

In [16]:
timeframe = ('1966-10-31' <= data.index) & (data.index <= '2016-11-30')
data = data[timeframe]

We then drop securities whose timeframes are less than 20 years.

In [17]:
# Get securities code
permno = data['permno'].unique()
clean_data = pd.DataFrame()

for permno in tqdm(permno):
    #print(label)
    content = data[data['permno']==permno]
    
    # Drop duplicate rows
    #content = content.loc[~content.index.duplicated(keep='first')]
    
    #if len(content)<(20*12): 
        #continue
        
    clean_data = pd.concat([clean_data, content], axis=0)

KeyboardInterrupt: 

In [ ]:
display(clean_data)

In [ ]:
print("Number of securities:", len(clean_data['permno'].unique()))

In [ ]:
print('missing entries: ' + str(data.isna().values.sum()))

In [ ]:
clean_data.to_csv('data_94_132_sic_4020_stock.csv')

In [ ]:
print(len(clean_data['sic2'].unique()))